In [11]:
import rdflib
import tkinter as tk
from owlready2 import *
def search():
    result_label.configure(text='hello')

win = tk.Tk()
win.title('Ser531 Project')
win.geometry('800x800')
win.configure(background='white')
search_panel = tk.Frame(win)
search_panel.pack(side=tk.TOP)
search_label = tk.Label(search_panel, text='Key/ID')
search_label.pack(side=tk.LEFT)
search_entry = tk.Entry(search_panel)
search_entry.pack(side=tk.LEFT)
search_btn = tk.Button(win, text='search', command=search)
search_btn.pack()
result_label = tk.Label(win)
result_label.pack()
win.mainloop()
    

In [9]:
import rdflib
import tkinter as tk
from owlready2 import *
class SparqlQueries:
    def __init__(self):
        my_world = World()
        my_world.get_ontology("http://localhost:8890/edit/SER531TEAM/owlRules.owl").load()
#         sync_reasoner(my_world)
        self.graph = "ddd"
        
    def search(self):
        print(self.graph)
#         query = "base <http://localhost:8890/edit/SER531TEAM/owlRules.owl>"\
#                 "SELECT ?s"\
#                 "WHERE{"\
#                 "?s rdf:about ?o ."\
#                 "}"
#         resultsList = self.graph.query(query)
#         print("hello")
#         return 1
runQuery = SparqlQueries()
runQuery.search()

OwlReadyOntologyParsingError: RDF/XML parsing error in file http://localhost:8890/login?next=%2Fedit%2FSER531TEAM%2FowlRules.owl, line 111, column 2.